# Test pipeline for processing PDFs and storing in ChromaDB

In [2]:
import sys
import os

# Add the project root to the Python path to allow imports from sibling directories
# This is necessary because the notebook is in a subfolder ('backend')
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Adjusted sys.path to include project root: {project_root}")

# Ensure llmAG is importable for the RAG pipeline + LLM factory
llm_root = os.path.join(project_root, 'llmAG')
if llm_root not in sys.path:
    sys.path.insert(0, llm_root)
    print(f"Adjusted sys.path to include LLM root: {llm_root}")


Adjusted sys.path to include project root: c:\Users\Noah\Documents\Workspace\GenAI


In [3]:
%load_ext autoreload
%autoreload 2

import glob
import json
import os
import textwrap
from typing import List, Dict, Any

import chromadb
from tqdm import tqdm

from embeddingModels.BaseEmbeddingModel import BaseEmbeddingModel
from embeddingModels.ModernBertEmbedder import ModernBertEmbedder
from embeddingModels.QwenEmbedder import QwenEmbedder
from pdfProcessing.doclingTest import setup_docling_converter, extract_sections_from_doc, extract_metadata

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Noah\Documents\Workspace\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up folders

In [4]:
CURRENT_MODEL = "bert"  # Select either qwen or bert
INPUT_FOLDER = "../data/testPDFs"
OUTPUT_FOLDER = "../data/testPDFOutput/pipelineTest"
CHROMA_DB_DIR = "./chroma_db"

In [5]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pdf_files = glob.glob(os.path.join(INPUT_FOLDER, "*.pdf"))
collection_names = {"bert": "scientific_papers_bert", "qwen": "scientific_papers_qwen"}

## Set up ChromaDB Client, Collection and Document Converter

In [6]:
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
pipeline_test_collection = chroma_client.get_or_create_collection(
    name=collection_names[CURRENT_MODEL],
    metadata={"hnsw:space": "ip"}
)
converter = setup_docling_converter()
# I used docling from IBM, can also describe images

2026-01-02 16:32:29,834 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


### Docling
https://www.docling.ai/

## Convert PDFs and store in json

In [7]:
%%time
for pdf_path in tqdm(pdf_files):
    file_stem = os.path.splitext(os.path.basename(pdf_path))[0]
    try:
        result = converter.convert(pdf_path)

        sections = extract_sections_from_doc(result.document)

        metadata = extract_metadata(sections)

        final_output = {
            "filename": os.path.basename(pdf_path),
            "metadata": metadata,
            "sections": sections
        }

        out_path = os.path.join(OUTPUT_FOLDER, f"{file_stem}_converted.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(final_output, f, indent=2)

        print(f"✅ Processed: {file_stem}")
        print(f"   found ID: {metadata.get('arxiv_id')}")
        print(f"   found {len(metadata.get('authors', []))} authors")

    except Exception as e:
        print(f"❌ Failed {file_stem}: {e}")


  0%|          | 0/3 [00:00<?, ?it/s]2026-01-02 16:32:30,265 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 16:32:32,033 - INFO - Going to convert document batch...
2026-01-02 16:32:32,035 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 98cea8ab80245d656e57fdc38b70960e
2026-01-02 16:32:32,056 - INFO - Loading plugin 'docling_defaults'
2026-01-02 16:32:32,060 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-02 16:32:32,077 - INFO - Loading plugin 'docling_defaults'
2026-01-02 16:32:32,085 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-02 16:32:32,822 - INFO - Accelerator device: 'cpu'
[INFO] 2026-01-02 16:32:32,852 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-01-02 16:32:32,870 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\Noah\Documents\Workspace\GenAI\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 

✅ Processed: Attention is all you need
   found ID: arXiv:1706.03762v7
   found 9 authors


2026-01-02 16:33:41,992 - INFO - Finished converting document BERT.pdf in 32.05 sec.
 67%|██████▋   | 2/3 [01:11<00:35, 35.20s/it]2026-01-02 16:33:42,013 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 16:33:42,016 - INFO - Going to convert document batch...
2026-01-02 16:33:42,016 - INFO - Processing document sentence bert.pdf


✅ Processed: BERT
   found ID: arXiv:1810.04805v2
   found 3 authors


2026-01-02 16:34:03,963 - INFO - Finished converting document sentence bert.pdf in 21.97 sec.
100%|██████████| 3/3 [01:33<00:00, 31.24s/it]

✅ Processed: sentence bert
   found ID: arXiv:1908.10084v1
   found 3 authors
CPU times: total: 8min 24s
Wall time: 1min 33s


In [8]:
del converter

## Embed and store in ChromaDB

In [9]:
def ingest_papers_to_chroma(
        json_folder: str,
        collection: chromadb.Collection,
        embedding_model: BaseEmbeddingModel
):
    """
    Reads structured JSON papers and ingests them into ChromaDB.
    """

    json_files = glob.glob(os.path.join(json_folder, "*.json"))
    print(f"Found {len(json_files)} JSON files to ingest.")

    for json_file in tqdm(json_files, desc="Processing Papers"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # --- A. Determine Parent ID ---
        # Prefer arXiv ID, fallback to filename if missing
        parent_id = data['metadata'].get('arxiv_id')
        if not parent_id:
            parent_id = data['filename']
            # TODO: get ID from sematic scholar

        # Clean ID (Chroma requires IDs to be strings, usually safe chars)
        parent_id = parent_id.replace(" ", "_").replace(":", "_")

        # --- B. Prepare Batches for this Document ---
        documents: List[str] = []
        metadatas: List[Dict[str, Any]] = []
        ids: List[str] = []

        global_meta = {
            "parent_id": parent_id,
            "filename": data['filename'],
            "title": data['metadata'].get('title', "Unknown"),
            "authors": ", ".join(data['metadata'].get('authors', [])),
            "arxiv_id": data['metadata'].get('arxiv_id', "N/A")
        }

        for section_header, content in tqdm(data['sections'].items(), desc="Processing Sections"):
            if not content.strip():
                continue

            # 1. Create Unique ID for this chunk
            safe_header = section_header.replace(" ", "_")[:50]
            chunk_id = f"{parent_id}#{safe_header}"

            # 2. Create Metadata for this chunk
            chunk_meta = global_meta.copy()
            chunk_meta["section"] = section_header
            chunk_meta["is_preamble"] = (section_header == "Preamble")

            # removing \n from content
            content = content.replace("\n", " ")
            documents.append(content)
            metadatas.append(chunk_meta)
            ids.append(chunk_id)

        # --- D. Generate Embeddings ---
        if documents:
            # Use your custom class to encode
            embeddings_np = embedding_model.encode(documents)
            # Convert numpy to python list for Chroma
            embeddings_list = embeddings_np.tolist()

            # --- E. Upsert to Chroma ---
            # using upsert handles re-runs gracefully (updates existing IDs)
            print('Generating embeddings')
            collection.upsert(
                ids=ids,
                embeddings=embeddings_list,
                documents=documents,
                metadatas=metadatas
            )

    print("Ingestion Complete.")

In [10]:
match CURRENT_MODEL:
    case "bert":
        embedder = ModernBertEmbedder(
            model_name="Alibaba-NLP/gte-modernbert-base",
            normalize=True
        )
    case "qwen":
        embedder = QwenEmbedder("Qwen/Qwen3-Embedding-8B", use_fp16=True)


Loading Alibaba-NLP/gte-modernbert-base on cpu...


In [11]:
%%time
ingest_papers_to_chroma(
    json_folder=OUTPUT_FOLDER,
    collection=pipeline_test_collection,
    embedding_model=embedder
)

Found 3 JSON files to ingest.


Processing Sections: 100%|██████████| 28/28 [00:00<?, ?it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Processing Papers:  33%|███▎      | 1/3 [00:45<01:31, 45.82s/it]

Generating embeddings


Processing Papers:  67%|██████▋   | 2/3 [02:29<01:20, 80.08s/it]

Generating embeddings


Processing Papers: 100%|██████████| 3/3 [03:00<00:00, 60.05s/it]

Generating embeddings
Ingestion Complete.
CPU times: total: 11min 35s
Wall time: 3min


In [12]:
print(f"Collection Count: {pipeline_test_collection.count()}")

Collection Count: 77


In [13]:
def query_chroma(
        collection: chromadb.Collection,
        query_text: str,
        model: BaseEmbeddingModel,
        n_results: int = 5,
):
    print(f"--- 🔍 Querying for: '{query_text}' ---")

    try:
        query_vector_np = model.encode([query_text])
        query_vector_list = query_vector_np.tolist()

        results = collection.query(
            query_embeddings=query_vector_list,
            n_results=n_results,
            # Optional: Filter by metadata (e.g., only from specific paper)
            # where={"parent_id": "arXiv:1706.03762v7"}
        )

        # 5. Display Results
        if not results['ids'][0]:
            print("No results found.")
            return

        print(f"\n✅ Found {len(results['ids'][0])} relevant chunks:\n")

        for i in range(len(results['ids'][0])):
            doc_id = results['ids'][0][i]
            score = results['distances'][0][i]  # Lower is better (L2 distance)
            content = results['documents'][0][i]
            metadata = results['metadatas'][0][i]

            print(f"Result #{i + 1} (Distance: {score:.4f})")
            print(f"📄 Paper: {metadata.get('title', 'Unknown')}")
            print(f"📌 Section: {metadata.get('section', 'Unknown')}")
            print(f"🔗 ID: {doc_id}")
            print("-" * 40)
            print("📝 Content Snippet:")
            print(textwrap.fill(content[:300] + "...", width=80))  # Preview first 300 chars
            print("\n" + "=" * 60 + "\n")

        return results
    except Exception as e:
        print(e)

In [14]:
results = query_chroma(
    collection=pipeline_test_collection,
    query_text="transformer architecture details",
    model=embedder,
    n_results=3
)

--- 🔍 Querying for: 'transformer architecture details' ---

✅ Found 3 relevant chunks:

Result #1 (Distance: 0.5367)
📄 Paper: Attention Is All You Need
📌 Section: 5 Training
🔗 ID: arXiv_1706.03762v7#5_Training
----------------------------------------
📝 Content Snippet:
This section describes the training regime for our models....


Result #2 (Distance: 0.5779)
📄 Paper: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
📌 Section: 4 Experiments
🔗 ID: arXiv_1810.04805v2#4_Experiments
----------------------------------------
📝 Content Snippet:
In this section, we present BERT fine-tuning results on 11 NLP tasks....


Result #3 (Distance: 0.6134)
📄 Paper: Attention Is All You Need
📌 Section: Scaled Dot-Product Attention
🔗 ID: arXiv_1706.03762v7#Scaled_Dot-Product_Attention
----------------------------------------
📝 Content Snippet:
MatMul SoftMax Mask (opt.) Scale MatMul K Figure 2: (left) Scaled Dot-Product
Attention. (right) Multi-Head Attention consists of 

## RAG: LLM answers from Chroma context


In [27]:
import os
import sys

# Ensure llmAG/rag is importable even if earlier cells were skipped
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

llm_root = os.path.join(project_root, "llmAG")
if llm_root not in sys.path:
    sys.path.insert(0, llm_root)



from langchain_core.documents import Document
from rag import BaseRetriever, RagPipeline

class ChromaRetriever(BaseRetriever):
    def __init__(self, collection: chromadb.Collection, model: BaseEmbeddingModel):
        self._collection = collection
        self._model = model

    def get_relevant_documents(self, query: str, k: int = 4) -> List[Document]:
        query_vector_np = self._model.encode([query])
        results = self._collection.query(
            query_embeddings=query_vector_np.tolist(),
            n_results=k,
        )

        ids = results.get("ids", [[]])[0] if results else []
        documents = results.get("documents", [[]])[0] if results else []
        metadatas = results.get("metadatas", [[]])[0] if results else []
        distances = results.get("distances", [[]])[0] if results else []

        docs: List[Document] = []
        for doc_id, text, meta, score in zip(ids, documents, metadatas, distances):
            meta_dict = dict(meta) if meta else {}
            meta_dict["score"] = score
            meta_dict["id"] = doc_id
            docs.append(Document(page_content=text, metadata=meta_dict))
        return docs


In [28]:
retriever = ChromaRetriever(pipeline_test_collection, embedder)
pipeline = RagPipeline(retriever, model='mistral-nemo', temperature=0.2)


In [30]:
'''questions = [
    "What does RAG combine, and why is it useful?",
    "What does top-k retrieval mean in this context?",
    "What chunk size and overlap ranges are suggested?",
    "Why does chunk overlap matter for retrieval?",
    "What should the assistant do when the answer is not in the context?",
]'''

questions = [
    "How does the Transformer in \"Attention Is All You Need\" replace recurrence and convolution, and what role does self-attention play in modeling dependencies?",
    "For BERTLARGE, how many layers, what hidden size, and how many attention heads are used?",
    "In BERT, what is the special classification token placed at the start of every input sequence?",
]

for q in questions:
    response = pipeline.run(q, k=3, include_sources=True)
    print("\n=== Question ===")
    print(q)
    print("\nAnswer:\n", response.answer)
    if response.sources:
        print("\nSources:")
        for idx, doc in enumerate(response.sources, start=1):
            title = doc.metadata.get('title', 'Unknown') if doc.metadata else 'Unknown'
            section = doc.metadata.get('section', 'Unknown') if doc.metadata else 'Unknown'
            score = doc.metadata.get('score') if doc.metadata else None
            score_str = f"{score:.4f}" if isinstance(score, (int, float)) else 'n/a'
            print(f"{idx}. {title} | {section} | score={score_str}")


2026-01-02 19:16:02,520 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



=== Question ===
How does the Transformer in "Attention Is All You Need" replace recurrence and convolution, and what role does self-attention play in modeling dependencies?

Answer:
 The Transformer in "Attention Is All You Need" replaces recurrence (like LSTMs or GRUs) and convolution by using self-attention mechanisms to model dependencies between input elements. Self-attention allows the model to selectively "attend" to different positions of a single sequence, enabling it to capture dependencies regardless of their distance. This is unlike recurrent models that process inputs sequentially, or convolutional models where the number of operations grows with the distance between positions. In the Transformer, the number of operations required to relate signals from two arbitrary input positions remains constant, making it easier to learn long-range dependencies. However, this comes at the cost of reduced effective resolution due to averaging attention-weighted positions, which is cou

KeyboardInterrupt: 